<a href="https://colab.research.google.com/github/michelli-ofig/DataScience_Em_Producao/blob/main/m07_v01_store_sales_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.0. Imports

In [105]:
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.7.10


In [106]:
import matplotlib.pyplot as plt
%matplotlib inline  

In [107]:
!pip install tabulate

In [108]:
!pip install inflection

In [109]:
!pip install boruta

In [111]:
import inflection
import math 
import seaborn as sns
import matplotlib.gridspec as gridSpec
import numpy as np
import datetime as dt
import pandas as pd
import timedelta
from tabulate              import tabulate
from datetime              import timedelta
from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder 
from sklearn.ensemble      import RandomForestRegressor
from sklearn.metrics       import mean_absolute_error, mean_squared_error
from scipy                 import stats as ss
from matplotlib            import pyplot as plt
from matplotlib            import rcParams
from numpy                 import int64
from IPython.display       import Image
from datetime              import datetime
from datetime              import timedelta
from boruta                import BorutaPy

## 0.1. Helper Functions

In [ ]:
def cramer_v (x,y):
    cm= pd.crosstab(x,y).values
    n=cm.sum()
    r, k = cm.shape
    
    chi2 = ss.chi2_contingency(cm)[0]
    chi2corr = max(0, chi2 - (k-1)*(r-1)/(n-1))
    
    kcorr = k - (k-1)**2/(n-1)
    rcorr = r - (r-1)**2/(n-1)
    return np.sqrt((chi2corr/n)/(min( kcorr-1, rcorr-1)))


## 0.2. Carregando os Dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df_sales_raw = pd.read_csv('/content/drive/MyDrive/repos/DataScience_Em_Producao/data/train.csv')
df_store_raw = pd.read_csv('/content/drive/MyDrive/repos/DataScience_Em_Producao/data/store.csv')





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df_sales_raw = pd.read_csv('/content/drive/MyDrive/repos/DataScience_Em_Producao/data/train.csv')
df_store_raw = pd.read_csv('/content/drive/MyDrive/repos/DataScience_Em_Producao/data/store.csv')





In [ ]:
df_store_raw.head()

In [ ]:
df_sales_raw.head()

In [ ]:
df_raw= pd.merge(df_sales_raw, df_store_raw, how='left', on='Store')



In [ ]:
df_sales_raw.head()

In [ ]:
df_raw=pd.merge(df_sales_raw, df_store_raw, how='left', on= 'Store' )

In [ ]:
df_raw.sample()

In [ ]:
print(df_raw.shape)

# 1 - Descrição dos dados

## 1.0 - Rename Columns

In [ ]:
df1=df_raw.copy()

In [ ]:
df1.columns

In [ ]:
cols_old= ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'] 

In [ ]:
snakecase = lambda x: inflection.underscore(x)  #Função para transformar os dados em snakecase
cols_new = list(map(snakecase, cols_old))

In [ ]:
df1.columns = cols_new

In [ ]:

df1.columns

## 1.2 Data Dimensions

In [ ]:
print('Number of Rows: {}' .format(df1.shape[0]))
print('Number of Cols: {}' .format(df1.shape[1]))

## 1.3 Data Type

In [ ]:
df1.dtypes


## 1.4 Check NA

In [ ]:
df1.isna().sum()

## 1.5 Fillout NA 

In [ ]:
#competition_distance              2642
#competition_open_since_month    323348
#competition_open_since_year     323348
#promo2_since_week               508031
#promo2_since_year               508031
#promo_interval                  508031

In [ ]:
#competition_distance 
df1['competition_distance'].max()

In [ ]:
df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 200000.0 if math.isnan(x)else x)

In [ ]:
print(df1[df1['competition_distance']==200000.0].shape)

In [ ]:
df1.isna().sum()

In [ ]:
df1['date']= pd.to_datetime(df1['date'])

In [ ]:
df1.dtypes

In [ ]:
df1.isna().sum()

In [ ]:
df1['competition_open_since_year']=df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x ['competition_open_since_year'], axis=1 )
df1['competition_open_since_month']=df1.apply(lambda x: x['date'].month if math.isnan(x ['competition_open_since_month']) else x['competition_open_since_month'], axis=1)   

In [ ]:
#promo2_since_week
df1['promo2_since_week']=df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x ['promo2_since_week'],axis=1 )

#promo2_since_year 
df1['promo2_since_year']=df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x ['promo2_since_year'], axis=1 )

#Dicionario criado para fazer o mapeamento para a coluna month_map
month_map = {1:'Jan',2:'Fev', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}

#Preenchendo os NA com 0
df1['promo_interval'].fillna(0, inplace=True)

#Criando a coluna month_map buscando a partir no mês da coluna data 
df1['month_map'] = df1['date'].dt.month.map(month_map)

#Criando a coluna in_promo: Se o mês da coluna month map estiver contido na coluna promo interval retorna 1(esta participando da promo), 
#se retornar 0 signigica que não esta participando da promoção.

df1['in promo']= df1[['promo_interval', 'month_map']].apply(lambda x: 0 if x['promo_interval'] == 0 
else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)
                                                            










In [ ]:
df1.sample(50).T

In [ ]:
df1.isna().sum()

In [ ]:
print(df1['in promo'].unique() )

In [ ]:
print(df1[df1['in promo']==1].shape)

In [ ]:
print(df1.shape)

## 1.6 Change dtypes

In [ ]:
df1.dtypes

In [ ]:
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype(int64)


In [ ]:
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype(int64)


In [ ]:
df1.dtypes

In [ ]:
df1['promo2_since_week'] = df1['promo2_since_week'].astype(int64)
df1['promo2_since_year'] = df1['promo2_since_year'].astype(int64)


In [ ]:
df1.dtypes

## 1.7 Descritive Statistical

In [ ]:
num_attributes = df1.select_dtypes(include = ['int64', 'float64'])
cat_attributes = df1.select_dtypes(exclude = ['int64', 'float64','datetime64[ns]'])

In [ ]:
cat_attributes.sample(2)

### 1.7.1 Numerical Attributes

In [ ]:
#Central Tendence - mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

#Dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(np.min)).T
d3 = pd.DataFrame(num_attributes.apply(np.max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min() )).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew()  )).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis() )).T

#concatenate

m = pd.concat([d2,d3,d4,ct1,ct2,d1,d5,d6] ).T.reset_index()
m.columns = ['atributes','min', 'max', 'range', 'mean', 'median', 'std', 'skew','kurtosis']
m




In [ ]:
sns.set(rc={'figure.figsize':(30,21.27)})
sns.displot(df1['competition_distance'])


### 1.7.2 Categorical Attributes

In [ ]:
cat_attributes.apply(lambda x: x.unique().shape[0]) #Imprime os tipos de cada variável

In [ ]:
df1.head()

In [ ]:
sns.set(rc={'figure.figsize':(21,15)})
aux1 = df1[df1['state_holiday'] != 0 & (df1['sales'] > 0)]
sns.boxplot ( x ='state_holiday', y= 'sales', data= aux1)

# 2.0 PASSO 02: FEATURE ENGINEERING

## 2.1 Mapa Mental de Hipóteses

In [ ]:
df2 = df1.copy()

In [ ]:
Image('img/MindMapHipothesis.png')


## Criação  das Hipóteses

## 2.1.1 Hipóteses Lojas

**1.** Lojas com maior quadro de funcionários deveriam vender mais.

**2.** Lojas com maior capacidade de estoque deveriam vender mais.

**3.** Lojas com maior porte deveriam vender mais.

**4.** Lojas com competidores mais próximos deveriam vender menos.

**5.** Lojas com competidores a mais tempos deveriam vender mais .

**6.** Lojas com maior sortimento deveriam vender mais.

## 2.1.2 Hipóteses Produtos


 **1.** Lojas que investem mais em Marketing deveriam vender mais.

**2.** Lojas que expõe mais os produtos nas vitrines deveriam vender mais.

**3.** Lojas que tem preços menores nos produtos deveriam vender mais.

**4.** Lojas como promoções mais agressivas (descontos maiores), deveriam vender mais.

**5.** Lojas como promoções ativas por mais tempo deveriam vender mais.

**6.** Lojas com mais dias de promoção deveriam vender mais 



## 2.1.3. Hipótese Tempo    


**1.** Lojas abertas durante o feriado do Natal deveriam vender mais.

**2.** Lojas  deveriam vender mais ao longo dos anos.

**3.** Lojas  deveriam vender mais no segundo semestre do ano.

**4.** Lojas  deveriam vender mais depois do dia 10 de cada mês.

**5.** Lojas  deveriam vender menos aos final de semana.

**6.** Lojas  deveriam vender menos durante os feriados escolares.

## 2.2 Lista final de Hipóteses

**1.** Lojas com competidores mais próximos deveriam vender menos.

**2.** Lojas com maior sortimento deveriam vender mais.

**3.** Lojas com competidores a mais tempos deveriam vender mais.

**4.** Lojas como promoções mais agressivas (descontos maiores), deveriam vender mais.

**5.** Lojas com promoções ativas por mais tempo deveriam vender mais.

**6.** Lojas com mais dias de promoção deveriam vender mais. 

**7.** Lojas abertas durante o feriado do Natal deveriam vender mais.

**8.** Lojas  deveriam vender mais ao longo dos anos.

**9.** Lojas  deveriam vender mais no segundo semestre do ano.

**10.** Lojas  deveriam vender mais depois do dia 10 de cada mês.

**11.** Lojas  deveriam vender menos aos finais de semana.

**12.** Lojas  deveriam vender menos durante os feriados escolares.





## 2.3 Feature Enginnering

In [ ]:
#year 
df2['year'] = df2['date'].dt.year

#month
df2['month'] = df2['date'].dt.month

#day
df2['day'] = df2['date'].dt.day

#week_of_year
df2['week_of_year'] = df2['date'].dt.isocalendar().week
#df2['week_of_year'] = df2['date'].dt.weekofyear

# year_week
df2['year_week'] = df2['date'].dt.strftime('%Y - %W')

# competition_since
df2['competition_since']= df2.apply( lambda x: dt.datetime( year=x['competition_open_since_year'], month=x['competition_open_since_month'],day=1 ), axis=1 )
df2['competition_time_month']= ((df2['date'] - df2['competition_since'])/30).apply(lambda x: x.days).astype(int)

# promo since
df2['promo_since'] = df2['promo2_since_year'].astype(str) + '-' + df2['promo2_since_week'].astype(str)
df2['promo_since'] = df2['promo_since'].apply(lambda x: dt.datetime.strptime(x + '-1','%Y-%W-%w') - dt.timedelta(days=7))
df2['promo_time_week'] =((df2['date'] - df2['promo_since'])/7).apply(lambda x: x.days).astype(int)

#assortment
df2['assortment']= df2['assortment'].apply(lambda x: 'basic' if x=='a' else 'extra' if x=='b' else 'extended')

#state_holiday
df2['state_holiday']=df2['state_holiday'].apply(lambda x: 'public_holiday' if x=='a' else 'easter_holiday' if x=='b' else 'christmas' if x=='c' else 'regular_day')





In [ ]:
df2.head().T

# 3.0 Filtragem de Imóveis 

In [ ]:
df3 = df2.copy()

In [ ]:
df3.head()

In [ ]:
df3 = df3[(df3['open']!=0)& (df3['sales']!=0)]

## 3.2. Seleção das Colunas

In [ ]:
cols_drop = ['customers','open', 'promo_interval', 'month_map']
df3 = df3.drop(cols_drop, axis=1)

# 4.0 - ANÁLISE EXPLORATÓRIA DOS DADOS

In [ ]:
df4 = df3.copy()

In [ ]:
df4.head()

## 4.1 Análise Univariada

### 4.1.2 Response Variable

In [ ]:
plt.rcParams['figure.figsize'] = (21,30)
sns.displot(df4['sales']);

### 4.1.2 Numerical Variable

In [ ]:
cat_attributes.head()

#state_holiday	
#store_type	
#assortment	


In [ ]:
sns.set(rc={'figure.figsize':(21,15)})
num_attributes.hist(bins =25);

### 4.1.3 Categorical Variable

In [ ]:
#state_holiday
plt.figure( figsize=(20, 18))
plt.subplot(3,2,1)

a = df4[df4['state_holiday']!= 'regular_day']
sns.countplot(a['state_holiday'])


plt.subplot(3,2,2)
sns.kdeplot(df4[df4['state_holiday'] == 'public_holiday']['sales'],label ='public_holiday', shade=True)
sns.kdeplot(df4[df4['state_holiday'] == 'easter_holiday']['sales'],label ='easter_holiday', shade=True)
sns.kdeplot(df4[df4['state_holiday'] == 'christmas']['sales'],label ='christmas', shade=True)

#store_type

plt.subplot(3,2,3)
sns.countplot(df4['store_type']);

plt.subplot(3,2,4)
sns.kdeplot(df4[df4['store_type'] == 'a']['sales'],label ='a', shade=True);
sns.kdeplot(df4[df4['store_type'] == 'b']['sales'],label ='b', shade=True);
sns.kdeplot(df4[df4['store_type'] == 'c']['sales'],label ='c', shade=True);
sns.kdeplot(df4[df4['store_type'] == 'd']['sales'],label ='d', shade=True);

#assortment

plt.subplot(3,2,5)
sns.countplot(df4['assortment']);

plt.subplot(3,2,6)
sns.kdeplot(df4[df4['assortment'] == 'basic']['sales'],label ='basic', shade=True);
sns.kdeplot(df4[df4['assortment'] == 'extended']['sales'],label ='extended', shade=True);
sns.kdeplot(df4[df4['assortment'] == 'extra']['sales'],label ='extra', shade=True);








## 4.2 Análise Bivariada

### H1. Lojas com maior sortimento deveriam vender mais.
***False*** Lojas com MAIOR SORTIMENTO vendem MENOS

In [ ]:
aux1= df4[['assortment', 'sales']].groupby('assortment').sum().reset_index()

aux2 = df4[['year_week','assortment','sales']].groupby(['year_week','assortment']).sum().reset_index()
aux2.pivot(index= 'year_week', columns='assortment', values='sales').plot();

aux3 = aux2[aux2['assortment']=='extra']
aux3.pivot(index= 'year_week', columns='assortment', values='sales').plot();

###  H2 - Lojas com competidores mais próximos deveriam vender menos.
**Falsa** Lojas com competidores mais PRÓXIMOS vendem MAIS






In [ ]:
aux1 = df4[['competition_distance', 'sales']].groupby('competition_distance').sum().reset_index()
plt.subplot(1,3,1)
sns.scatterplot(x ='competition_distance', y='sales', data=aux1)

plt.subplot(1,3,2)
bins = list(np.arange(0,20000, 1000))
aux1['competition_distance_binned'] = pd.cut(aux1['competition_distance'], bins=bins)
aux2 = aux1[['competition_distance_binned', 'sales']].groupby('competition_distance_binned').sum().reset_index()
sns.barplot(x='competition_distance_binned', y='sales', data=aux2);
plt.xticks(rotation=90)

plt.subplot(1,3,3)
sns.heatmap(aux1.corr(method='pearson'), annot=True)

### H3. Lojas com competidores a mais tempo deveriam vender mais.
**Falsa** Loja com COMPETIDORES a MAIS TEMPO vende MENOS. 

In [ ]:
plt.figure( figsize=(40, 22))

plt.subplot(1,3,1)
aux1 = df4[['competition_time_month', 'sales']].groupby('competition_time_month').sum().reset_index()
aux2 = aux1[(aux1['competition_time_month']<120)& (aux1['competition_time_month'] !=0)]
sns.barplot(x='competition_time_month', y='sales', data=aux2);
plt.xticks(rotation=90);

plt.subplot(1,3,2)
sns.regplot(x='competition_time_month', y='sales', data=aux2)

plt.subplot(1,3,3)
x=sns.heatmap(aux1.corr(method='pearson'), annot =True)




### **H4.**  Lojas como promoções ativas por mais tempo deveriam vender mais.
**Falsa** Lojas com promoçoes ativas por MAIS TEMPO vendem MENOS, depois de um certo período de promoção.

In [ ]:
aux1 = df4[['promo_time_week', 'sales']].groupby('promo_time_week').sum().reset_index()

plt.subplot(2,3,1)
aux2 = aux1[aux1['promo_time_week'] > 0]  #promo extendido
sns.barplot(x='promo_time_week', y='sales', data=aux2);
plt.xticks(rotation =90);

plt.subplot(2,3,2)
sns.regplot(x='promo_time_week', y='sales', data=aux2);

plt.subplot(2,3,3)
aux3 = aux1[aux1['promo_time_week'] < 0]  #promo regular 
sns.barplot(x='promo_time_week', y='sales', data=aux3);
plt.xticks(rotation =90);

plt.subplot(2,3,4)
sns.regplot(x='promo_time_week', y='sales', data=aux3);

plt.subplot(2,3,5)
sns.heatmap(aux1.corr(method='pearson'), annot=True);


### <s>**H5.** Lojas com mais dias de promoção deveriam vender mais </s>


### **H6.** Lojas como promoções consecutivas deveriam vender mais.
**Falsa** Lojas como promoções consecutivas vendem menos.

In [ ]:
df4[['promo','promo2', 'sales']].groupby(['promo','promo2']).sum().reset_index()

In [ ]:
aux1 = df4[(df4['promo']==1) & (df4['promo2']==1)][['year_week','sales']].groupby('year_week').sum().reset_index()
ax = aux1.plot()

aux2 = df4[(df4['promo']==1) & (df4['promo2']==0)][['year_week','sales']].groupby('year_week').sum().reset_index()
aux2.plot(ax=ax)

ax.legend(labels=['Tradicional & Extendida','Extendida']);

### **H7.** Lojas abertas durante o feriado do Natal deveriam vender mais.
**Falsa** Lojas abertas durante o feriado do Natal vendem menos.

In [ ]:
plt.figure( figsize=(20, 10))
aux= df4[df4['state_holiday'] != 'regular_day']

plt.subplot(1,2,1)
aux1 = aux[['state_holiday', 'sales']].groupby('state_holiday').sum().reset_index()
sns.barplot(x='state_holiday', y='sales', data = aux1);

plt.subplot(1,2,2)
aux2= aux[['year', 'state_holiday','sales']].groupby(['year', 'state_holiday']).sum().reset_index()
sns.barplot(x='year', y='sales', hue='state_holiday', data=aux2);   

### **H8.** Lojas  deveriam vender mais ao longo dos anos.
**Falsa** As lojas têm vendido menos ao longo dos anos.

In [ ]:
aux1 = df4[['year','sales']].groupby('year').sum().reset_index()
plt.figure( figsize=(20, 10))

plt.subplot(1,3,1)
sns.barplot(x='year', y='sales', data=aux1);

plt.subplot(1,3,2)
sns.regplot(x='year', y='sales', data=aux1);

plt.subplot(1,3,3)
sns.heatmap(aux1.corr(method='pearson'), annot=True);




### **H9.** Lojas  deveriam vender mais no segundo semestre do ano.
**Falsa** As vendas caem no segundo semestre.

In [ ]:
aux1 = df4[['month','sales']].groupby('month').sum().reset_index()
plt.figure( figsize=(20, 10))

plt.subplot(1,3,1)
sns.barplot(x='month', y='sales', data=aux1);

plt.subplot(1,3,2)
sns.regplot(x='month', y='sales', data=aux1);

plt.subplot(1,3,3)
sns.heatmap(aux1.corr(method='pearson'), annot=True);




### **H10.** Lojas  deveriam vender mais depois do dia 10 de cada mês.
**Verdadeira** Lojas vendem mais depois dia 10

In [ ]:
aux1 = df4[['day','sales']].groupby('day').sum().reset_index()
plt.figure( figsize=(20, 10))

plt.subplot(2,2,1)
sns.barplot(x='day', y='sales', data=aux1);

plt.subplot(2,2,2)
sns.regplot(x='day', y='sales', data=aux1);

plt.subplot(2,2,3)
sns.heatmap(aux1.corr(method='pearson'), annot=True);

aux1['before_after'] = aux1['day'].apply(lambda x: 'before_10_days' if x<=10 else 'after_10_days')
aux2= aux1[['before_after', 'sales']].groupby('before_after').sum().reset_index()

plt.subplot(2,2,4)
sns.barplot(x='before_after', y='sales', data= aux2)

### **H11.** Lojas  deveriam vender menos aos finais de semana.
**Falsa** Lojas vendem menos aos finais de semana.


In [ ]:
aux1 = df4[['day_of_week','sales']].groupby('day_of_week').sum().reset_index()
plt.figure( figsize=(20, 10))

plt.subplot(1,3,1)
sns.barplot(x='day_of_week', y='sales', data=aux1);

plt.subplot(1,3,2)
sns.regplot(x='day_of_week', y='sales', data=aux1);

plt.subplot(1,3,3)
sns.heatmap(aux1.corr(method='pearson'), annot=True);



### **H12.** Lojas  deveriam vender menos durante os feriados escolares.
**Verdadeira** Lojas vendem menos aos finais de semana, exceto nos meses de julho e agosto.

In [ ]:
aux1= df4[['school_holiday','sales']].groupby('school_holiday').sum().reset_index()
plt.subplot(2,1,1)
sns.barplot(x='school_holiday', y='sales', data=aux1);
aux2 =df4[['month','school_holiday','sales']].groupby(['month','school_holiday']).sum().reset_index()
plt.subplot(2,1,2)
sns.barplot(x='month', y='sales', hue='school_holiday', data=aux2);



### 4.2.1. Resumo da Hipóteses

In [ ]:
tab = [['Hipóteses', 'Conclusão', 'Relevãncia'],
['H1', 'Falsa', 'Baixa'],
['H2', 'Falsa', 'Média'],
['H3', 'Falsa', 'Média'],
['H4', 'Falsa', 'Baixa'],
['H5',   '-',     '-'],
['H6', 'Falsa', 'Baixa'],
['H7', 'Falsa', 'Média'],
['H8', 'Falsa', 'Alta'],
['H9', 'Falsa', 'Alta'],
['H10','Verdadeira', 'Alta'],
['H11','Verdadeira', 'Alta'],
['H12','Verdadeira', 'Baixa'],
]
print(tabulate(tab, headers='firstrow'))

## 4.3 - Análise Multivariada 

### 4.3.1. Numerical Attributes

In [ ]:
plt.figure( figsize=(20, 5))
correlation = num_attributes.corr(method='pearson')
sns.heatmap(correlation, annot=True)

### 4.3.2. Categorical Attibutes

In [ ]:
#Only categorical data
a=df4.select_dtypes(include='object')

#Calculante cramer V 
a1 = cramer_v(a['state_holiday'], a['state_holiday'])
a2 = cramer_v(a['state_holiday'], a['store_type'])
a3 = cramer_v(a['state_holiday'], a['assortment'])

a4 = cramer_v(a['store_type'], a['state_holiday'])
a5 = cramer_v(a['store_type'], a['store_type'])
a6 = cramer_v(a['store_type'], a['assortment'])

a7 = cramer_v(a['assortment'], a['state_holiday'])
a8 = cramer_v(a['assortment'], a['store_type'])
a9 = cramer_v(a['assortment'], a['assortment'])

#Final dataset
d = pd.DataFrame({'state_holiday':[a1,a2,a3],
              'store_type':[a4,a5,a6],
              'assortment':[a7,a8,a9], })
d = d.set_index(d.columns)
plt.figure( figsize=(20, 5))
sns.heatmap(d, annot=True);      

# 5.0. Data Preparation

In [ ]:
df5= df4.copy()

## 5.1. Normalização

## 5.2. Rescaling

In [ ]:
a= df5.select_dtypes(include=['int64','float64'])

In [ ]:
rs = RobustScaler()
mms = MinMaxScaler()

#competition distance 
df5['competition_distance']= rs.fit_transform(df5[['competition_distance']].values)

#competition time month
df5['competition_time_month']= rs.fit_transform(df5[['competition_time_month']].values)

#year
df5['year']= mms.fit_transform(df5[['year']].values)

#promo time week
df5['promo_time_week']= mms.fit_transform(df5[['promo_time_week']].values)

## 5.3. Transformação





## 5.3.1. Encoding

In [ ]:
# state_holiday - One Hot Encoding
df5= pd.get_dummies(df5, prefix =['state_holiday'], columns=['state_holiday'])

# store_type  - Label Enconding
le = LabelEncoder()
df5['store_type'] = le.fit_transform(df5['store_type'])

# assortment - Ordinal Encoding   - Transformou a variavél object em int  
assortment_dict = {'basic': 1, 'extra': 2, 'extended': 3}
df5['assortment'] = df5['assortment'].map(assortment_dict)

In [ ]:
df5.head()


## 5.3.2. Response Variable Transformation

In [ ]:
df5['sales'] = np.log1p(df5['sales'])

In [ ]:
sns.distplot(df5['sales']);

## 5.3.3. Nature Transformation





In [ ]:
#day_of_week
df5['day_of_week_sin']= df5['day_of_week'].apply(lambda x: np.sin(x * (2. * np.pi/7) ) )
df5['day_of_week_cos']= df5['day_of_week'].apply(lambda x: np.cos(x * (2. * np.pi/7) ) )

#month
df5['month_sin']= df5['month'].apply(lambda x: np.sin(x * (2. * np.pi/12) ) )
df5['month_cos']= df5['month'].apply(lambda x: np.cos(x * (2. * np.pi/12) ) )

#week_of_year
df5['week_of_year_sin']= df5['week_of_year'].apply(lambda x: np.sin(x * (2. * np.pi/52) ) )
df5['week_of_year_cos']= df5['week_of_year'].apply(lambda x: np.cos(x * (2. * np.pi/52) ) )

#day
df5['day_sin']= df5['day'].apply(lambda x: np.sin(x * (2. * np.pi/30) ) )
df5['day_cos']= df5['day'].apply(lambda x: np.cos(x * (2. * np.pi/30) ) )



In [ ]:
df5.head()

#6.0. Feature Selection

In [ ]:
df6=df5.copy()

##6.1. Split dataframe into training and test dataset


In [ ]:
df6.head()

##6.2. Boruta as Feature Selector

In [ ]:
cols_drop = ['week_of_year', 'day', 'month', 'day_of_week', 'promo_since', 'competition_since', 'year_week']
df6 = df6.drop(cols_drop, axis=1)

In [ ]:
df6[['store','date']].groupby('store').max().reset_index()['date'] [0] - timedelta(days=6*7)



In [ ]:
#Training dataset
X_train = df6[df6['date'] < '2015-06-19']
y_train = X_train['sales']

#Test dataset
X_test = df6[df6['date']>= '2015-06-19']
y_test = X_test['sales']

print('Training Min Date: {}'.format(X_train['date'].min() ))
print('Training Max Date: {}'.format(X_train['date'].max() ))

print('\nText Min Date: {}'.format(X_test['date'].min() ))
print('Text Max Date: {}'.format(X_test['date'].max() ))


In [ ]:
#training and test dataset boruta 
X_train_n = X_train.drop(['date', 'sales'], axis=1).values 
y_train_n = y_train.values.ravel()


#define RangdomForestRegressor
#rf = RandomForestRegressor(n_jobs=-1)

#define boruta
#boruta = BorutaPy( rf, n_estimators='auto', verbose=2, random_state=42 ).fit(X_train_n, y_train_n)

In [ ]:
#cols_selected = boruta.support_.tolist()
#best features
#X_train_fs = X_train.drop(['date','sales'], axis=1)
#cols_selected_boruta = X_train_fs.iloc[:, cols_selected ].columns.to_list()

#not selected boruta
#cols_not_selected_boruta = list(np.setdiff1d(X_train_fs.columns, cols_selected_boruta))

In [ ]:
#cols_selected_boruta

## 6.3. Manual Feature Selection

In [1]:
cols_selected_boruta = [
  'store',
  'promo',
  'store_type',
  'assortment',
  'competition_distance',
  'competition_open_since_month',
  'competition_open_since_year',
  'promo2',
  'promo2_since_week',
  'promo2_since_year',
  'competition_time_month',
  'promo_time_week',
  'day_of_week_sin',
  'day_of_week_cos',
  'month_sin',
  'month_cos',
  'week_of_year_sin',
  'week_of_year_cos',
  'day_sin',
  'day_cos']

  #columns to add
#feat_to_add = ['date','sales']

  #final features 
#cols_selected_boruta.extend(feat_to_add)


In [ ]:
cols_selected_boruta


# 7.0. Machine Learning Modelling

In [112]:
x_train = X_train[cols_selected_boruta]
x_test = X_test[cols_selected_boruta]

##7.1. Average Model 

In [116]:
#from sklearn.metrics       import mean_absolute_error, mean_squared_error

def mean_absolute_percentage_error(y,yhat):
    return np.mean(np.abs((y-yhat)/y))

def ml_error(model_name, y ,yhat):
    mae = mean_absolute_error(y,yhat)
    mape = mean_absolute_percentage_error(y,yhat)
    rmse = np.sqrt(mean_squared_error(y,yhat))

    return pd.DataFrame({'Model Name': model_name,
                         'MAE': mae,
                         'MAPE': mape,
                         'RMSE': rmse}, index=[0])

In [117]:
aux1 = x_test.copy()
aux1['sales'] = y_test.copy()

#prediction 
aux2 = aux1[['store','sales']].groupby('store').mean().reset_index().rename(columns={'sales': 'predictions'})
aux1 = pd.merge( aux1,aux2, how='left', on='store')
yhat_baseline = aux1['predictions']

#performance 
baseline_result = ml_error ('Average Model', np.expm1(y_test), np.expm1(yhat_baseline))
baseline_result

,Model Name,MAE,MAPE,RMSE
0,Average Model,1354.800353,0.455051,1835.135542


##7.2. Linear Regression Model



##7.3. Linear Regression Regularized Model